In [1]:
import os
import re
import json
import gzip
import codecs
import math
from math import log, ceil
import numpy as np
import modisco
import modisco.tfmodisco_workflow.workflow
from modisco.tfmodisco_workflow import workflow
import h5py
import pandas as pd
import modisco.util
from collections import Counter
from modisco.visualization import viz_sequence
import modisco.affinitymat.core
import modisco.cluster.phenograph.core
import modisco.cluster.phenograph.cluster
import modisco.cluster.core
import modisco.aggregator
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr, pearsonr, gaussian_kde
font = {'weight' : 'bold', 'size'   : 14}

TF-MoDISco is using the TensorFlow backend.
/users/amr1/miniconda3/envs/basepair/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/users/amr1/miniconda3/envs/basepair/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/users/amr1/miniconda3/envs/basepair/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In [2]:
data_dir = "/oak/stanford/groups/akundaje/amr1/pho4_final/lite_data/in-vitro/yeast/"
cbf1_filename = "GSM4980359_1uMCbf1_His_alldata.txt.gz"
pho4_filename = "GSM4980362_400nMPho4_GST_alldata.txt.gz"

pho4_seqToDdg = {}
firstLine = True
with gzip.open(data_dir+pho4_filename, 'rt') as inp:
    for line in inp:
        if firstLine:
            firstLine = False
            continue
        curr = line.strip().split('\t')[4]
        if curr == "_": continue
        if curr[36:] != "GTCTTGATTCGCTTGACGCTGCTG": print("Exception: ", curr)
        seq = curr[:36]
        if seq not in pho4_seqToDdg:
            pho4_seqToDdg[seq] = []
        pho4_seqToDdg[seq].append(log(float(line.strip().split('\t')[-1])))
        
cbf1_seqToDdg = {}
firstLine = True
with gzip.open(data_dir+cbf1_filename, 'rt') as inp:
    for line in inp:
        if firstLine:
            firstLine = False
            continue
        curr = line.strip().split('\t')[4]
        if curr == "_": continue
        if curr[36:] != "GTCTTGATTCGCTTGACGCTGCTG": print("Exception: ", curr)
        seq = curr[:36]
        if seq not in cbf1_seqToDdg:
            cbf1_seqToDdg[seq] = []
        cbf1_seqToDdg[seq].append(log(float(line.strip().split('\t')[-1])))

seqs = []
pho4_seqToLabel = {}
cbf1_seqToLabel = {}
for seq in pho4_seqToDdg:
    seqs.append(seq)
    pho4_seqToLabel[seq] = np.mean(pho4_seqToDdg[seq])
    cbf1_seqToLabel[seq] = np.mean(cbf1_seqToDdg[seq])

In [3]:
complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'} 
def getRevComp(seq):  # reverse complement function
    ret = ""
    for bp in seq.upper(): ret += complement[bp]
    return ret[::-1]

def generate_matrix(seq):
    seq_matrix = np.zeros((len(seq), 4))
    for j in range(len(seq)):
        if seq[j] == 'A':
            seq_matrix[j,0] = 1
        elif seq[j] == 'C':
            seq_matrix[j,1] = 1
        elif seq[j] == 'G':
            seq_matrix[j,2] = 1
        elif seq[j] == 'T':
            seq_matrix[j,3] = 1
    return seq_matrix

def get_PWM_max_score(sequence, score_matrix):
    score_len = score_matrix.shape[0]
    scores = []
    for j in range(len(sequence) - score_len + 1):
        seq_matrix = generate_matrix(sequence[j:j+score_len])
        scores.append(np.sum(score_matrix * seq_matrix))
    rc_sequence = getRevComp(sequence)
    for j in range(len(rc_sequence) - score_len + 1):
        seq_matrix = generate_matrix(rc_sequence[j:j+score_len])
        scores.append(np.sum(score_matrix * seq_matrix))
    return max(scores)

In [4]:
pred_dir = "/oak/stanford/groups/akundaje/amr1/pho4_final/lite_data/preds/"
obj_text1 = codecs.open(pred_dir+"pho4_nexus_gcpbm.json", 'r', encoding='utf-8').read()
pho4_seqToDeltaLogCounts = json.loads(obj_text1)
obj_text2 = codecs.open(pred_dir+"cbf1_nexus_gcpbm.json", 'r', encoding='utf-8').read()
cbf1_seqToDeltaLogCounts = json.loads(obj_text2)

pho4_pwm_scores = {}
cbf1_pwm_scores = {}
pho4_pwm_scores["distill"] = {}
cbf1_pwm_scores["distill"] = {}
pho4_pwm_scores["distill_1seq"] = {}
cbf1_pwm_scores["distill_1seq"] = {}
pho4_pwm_scores["distill_2seq"] = {}
cbf1_pwm_scores["distill_2seq"] = {}
pho4_pwm_scores["distill_5seq"] = {}
cbf1_pwm_scores["distill_5seq"] = {}
pho4_pwm_scores["distill_10seq"] = {}
cbf1_pwm_scores["distill_10seq"] = {}
pho4_pwm_scores["distill_20seq"] = {}
cbf1_pwm_scores["distill_20seq"] = {}
pho4_pwm_scores["distill_50seq"] = {}
cbf1_pwm_scores["distill_50seq"] = {}
for seq in seqs:
    pho4_pwm_scores["distill"][seq] = float(pho4_seqToDeltaLogCounts[seq][-1])
    cbf1_pwm_scores["distill"][seq] = float(cbf1_seqToDeltaLogCounts[seq][-1])
    pho4_pwm_scores["distill_1seq"][seq] = np.mean(np.array(pho4_seqToDeltaLogCounts[seq][1][:1]).astype(np.float) -
                                                   np.array(pho4_seqToDeltaLogCounts[seq][0][:1]).astype(np.float))
    cbf1_pwm_scores["distill_1seq"][seq] = np.mean(np.array(cbf1_seqToDeltaLogCounts[seq][1][:1]).astype(np.float) -
                                                   np.array(cbf1_seqToDeltaLogCounts[seq][0][:1]).astype(np.float))
    pho4_pwm_scores["distill_2seq"][seq] = np.mean(np.array(pho4_seqToDeltaLogCounts[seq][1][:2]).astype(np.float) -
                                                   np.array(pho4_seqToDeltaLogCounts[seq][0][:2]).astype(np.float))
    cbf1_pwm_scores["distill_2seq"][seq] = np.mean(np.array(cbf1_seqToDeltaLogCounts[seq][1][:2]).astype(np.float) -
                                                   np.array(cbf1_seqToDeltaLogCounts[seq][0][:2]).astype(np.float))
    pho4_pwm_scores["distill_5seq"][seq] = np.mean(np.array(pho4_seqToDeltaLogCounts[seq][1][:5]).astype(np.float) -
                                                   np.array(pho4_seqToDeltaLogCounts[seq][0][:5]).astype(np.float))
    cbf1_pwm_scores["distill_5seq"][seq] = np.mean(np.array(cbf1_seqToDeltaLogCounts[seq][1][:5]).astype(np.float) -
                                                   np.array(cbf1_seqToDeltaLogCounts[seq][0][:5]).astype(np.float))
    pho4_pwm_scores["distill_10seq"][seq] = np.mean(np.array(pho4_seqToDeltaLogCounts[seq][1][:10]).astype(np.float) -
                                                   np.array(pho4_seqToDeltaLogCounts[seq][0][:10]).astype(np.float))
    cbf1_pwm_scores["distill_10seq"][seq] = np.mean(np.array(cbf1_seqToDeltaLogCounts[seq][1][:10]).astype(np.float) -
                                                   np.array(cbf1_seqToDeltaLogCounts[seq][0][:10]).astype(np.float))
    pho4_pwm_scores["distill_20seq"][seq] = np.mean(np.array(pho4_seqToDeltaLogCounts[seq][1][:20]).astype(np.float) -
                                                   np.array(pho4_seqToDeltaLogCounts[seq][0][:20]).astype(np.float))
    cbf1_pwm_scores["distill_20seq"][seq] = np.mean(np.array(cbf1_seqToDeltaLogCounts[seq][1][:20]).astype(np.float) -
                                                   np.array(cbf1_seqToDeltaLogCounts[seq][0][:20]).astype(np.float))
    pho4_pwm_scores["distill_50seq"][seq] = np.mean(np.array(pho4_seqToDeltaLogCounts[seq][1][:50]).astype(np.float) -
                                                   np.array(pho4_seqToDeltaLogCounts[seq][0][:50]).astype(np.float))
    cbf1_pwm_scores["distill_50seq"][seq] = np.mean(np.array(cbf1_seqToDeltaLogCounts[seq][1][:50]).astype(np.float) -
                                                   np.array(cbf1_seqToDeltaLogCounts[seq][0][:50]).astype(np.float))

In [5]:
lite_dir = "/oak/stanford/groups/akundaje/amr1/pho4_final/lite_data/in-vivo/"
filepath1 = lite_dir+"pho4_nexus/100_around_summits.fa.matrix.w2"
PWMs = []
lines = []
for idx, line in enumerate(open(filepath1)):
    if line[0] == ">":
        if len(lines) == 4:
            PWMs.append(np.array(lines)[:,1:].astype('float').T)
            lines = []
            if len(PWMs) == 10: break
    else: lines.append(line.rstrip().split('\t'))
# Pho4 nexus: weeder_max_1_metadata.json
pho4_pwm = PWMs[1]

filepath2 = lite_dir+"cbf1_nexus/100_around_summits.fa.matrix.w2"
PWMs = []
lines = []
for idx, line in enumerate(open(filepath1)):
    if line[0] == ">":
        if len(lines) == 4:
            PWMs.append(np.array(lines)[:,1:].astype('float').T)
            lines = []
            if len(PWMs) == 10: break
    else: lines.append(line.rstrip().split('\t'))
# Cbf1 nexus: weeder_max_0_metadata.json
cbf1_pwm = PWMs[0]

pho4_pwm_scores["weeder"] = {}
cbf1_pwm_scores["weeder"] = {}
for seq in seqs:
    pho4_pwm_scores["weeder"][seq] = get_PWM_max_score(seq, pho4_pwm)
    cbf1_pwm_scores["weeder"][seq] = get_PWM_max_score(seq, cbf1_pwm)

In [6]:
lite_dir = "/oak/stanford/groups/akundaje/amr1/pho4_final/lite_data/in-vivo/"
filepath = lite_dir+"pho4_nexus/streme_out/streme.txt"
reading = False
PWMs = []
for idx, line in enumerate(open(filepath)):
    if "letter-probability" in line:
        reading = True
        width = int(line.rstrip().split(' ')[5])
        lines = []
        continue
    if reading:
        lines.append(line.rstrip().split(' '))
        width -= 1
        if width == 0:
            reading = False
            prob_mat = np.array(lines)[:,1:].astype('float')
            pwm = np.log2((prob_mat/0.25)+1e-4)
            PWMs.append(pwm)
            if len(PWMs) == 10: break
# Pho4 nexus: streme_max_0_metadata.json
pho4_pwm = PWMs[0]

lite_dir = "/oak/stanford/groups/akundaje/amr1/pho4_final/lite_data/in-vivo/"
filepath = lite_dir+"cbf1_nexus/streme_out/streme.txt"
reading = False
PWMs = []
for idx, line in enumerate(open(filepath)):
    if "letter-probability" in line:
        reading = True
        width = int(line.rstrip().split(' ')[5])
        lines = []
        continue
    if reading:
        lines.append(line.rstrip().split(' '))
        width -= 1
        if width == 0:
            reading = False
            prob_mat = np.array(lines)[:,1:].astype('float')
            pwm = np.log2((prob_mat/0.25)+1e-4)
            PWMs.append(pwm)
            if len(PWMs) == 10: break
# Cbf1 nexus: streme_max_0_metadata.json
cbf1_pwm = PWMs[0]

pho4_pwm_scores["streme"] = {}
cbf1_pwm_scores["streme"] = {}
for seq in seqs:
    pho4_pwm_scores["streme"][seq] = get_PWM_max_score(seq, pho4_pwm)
    cbf1_pwm_scores["streme"][seq] = get_PWM_max_score(seq, cbf1_pwm)

In [7]:
# MoDISco Lite
filename = "/oak/stanford/groups/akundaje/amr1/pho4_final/models/modisco-lite/pho4_nexus/modisco_counts_results.h5"
f = h5py.File(filename, 'r')
pattern_list = len(f['pos_patterns'])

def trim_motif(cwm_fwd):
    trim_threshold=0.3
    score_fwd = np.sum(np.abs(cwm_fwd), axis=1)
    trim_thresh_fwd = np.max(score_fwd) * trim_threshold
    pass_inds_fwd = np.where(score_fwd >= trim_thresh_fwd)[0]
    start_fwd, end_fwd = max(np.min(pass_inds_fwd) - 4, 0), min(np.max(pass_inds_fwd) + 4 + 1, len(score_fwd) + 1)
    trimmed_cwm_fwd = cwm_fwd[start_fwd:end_fwd]
    return trimmed_cwm_fwd

CWMs = []
for idx in range(min(10, pattern_list)):
    cwm = trim_motif(f['pos_patterns']['pattern_'+str(idx)]['contrib_scores'])
    CWMs.append(cwm)
# Pho4 nexus: modisco_max_0_metadata.json
pho4_pwm = CWMs[0]

filename = "/oak/stanford/groups/akundaje/amr1/pho4_final/models/modisco-lite/cbf1_nexus/modisco_counts_results.h5"
f = h5py.File(filename, 'r')
pattern_list = len(f['pos_patterns'])

def trim_motif(cwm_fwd):
    trim_threshold=0.3
    score_fwd = np.sum(np.abs(cwm_fwd), axis=1)
    trim_thresh_fwd = np.max(score_fwd) * trim_threshold
    pass_inds_fwd = np.where(score_fwd >= trim_thresh_fwd)[0]
    start_fwd, end_fwd = max(np.min(pass_inds_fwd) - 4, 0), min(np.max(pass_inds_fwd) + 4 + 1, len(score_fwd) + 1)
    trimmed_cwm_fwd = cwm_fwd[start_fwd:end_fwd]
    return trimmed_cwm_fwd

CWMs = []
for idx in range(min(10, pattern_list)):
    cwm = trim_motif(f['pos_patterns']['pattern_'+str(idx)]['contrib_scores'])
    CWMs.append(cwm)
# Cbf1 nexus: modisco_max_0_metadata.json
cbf1_pwm = CWMs[0]

pho4_pwm_scores["modisco"] = {}
cbf1_pwm_scores["modisco"] = {}
for seq in seqs:
    pho4_pwm_scores["modisco"][seq] = get_PWM_max_score(seq, pho4_pwm)
    cbf1_pwm_scores["modisco"][seq] = get_PWM_max_score(seq, cbf1_pwm)

In [8]:
class CalibratorFactory(object):
    def __call__(self, valid_preacts, valid_labels):
        raise NotImplementedError()

class LinearRegression(CalibratorFactory):
    def __init__(self, verbose=True):
        self.verbose = verbose 

    def __call__(self, valid_preacts, valid_labels):
        lr = LR().fit(valid_preacts.reshape(-1, 1), valid_labels)
    
        def calibration_func(preact):
            return lr.predict(preact.reshape(-1, 1))

        return calibration_func
    
num_samples = min(1000, ceil(0.1*len(seqs)))
print(num_samples, len(seqs))
calibration_samples = np.random.choice(seqs, num_samples, replace=False)

pho4_seqToPred = {}
cbf1_seqToPred = {}
for key in pho4_pwm_scores:
    pho4_sample_labels = []
    cbf1_sample_labels = []
    pho4_sample_preds = []
    cbf1_sample_preds = []
    for seq in calibration_samples:
        pho4_sample_labels.append(pho4_seqToLabel[seq])
        cbf1_sample_labels.append(cbf1_seqToLabel[seq])
        pho4_sample_preds.append(pho4_pwm_scores[key][seq])
        cbf1_sample_preds.append(cbf1_pwm_scores[key][seq])
    pho4_sample_labels = np.array(pho4_sample_labels)
    cbf1_sample_labels = np.array(cbf1_sample_labels)    
    pho4_sample_preds = np.array(pho4_sample_preds)
    cbf1_sample_preds = np.array(cbf1_sample_preds)
    
    lr1 = LinearRegression()
    lr2 = LinearRegression()
    calibration_func1 = lr1(pho4_sample_preds, pho4_sample_labels)
    calibration_func2 = lr2(cbf1_sample_preds, cbf1_sample_labels)
    
    pho4_seqToPred[key] = {}
    cbf1_seqToPred[key] = {}
    for seq in seqs:
        pho4_seqToPred[key][seq] = calibration_func1(np.array([pho4_pwm_scores[key][seq]]))[0]
        cbf1_seqToPred[key][seq] = calibration_func2(np.array([cbf1_pwm_scores[key][seq]]))[0]

1000 20414


In [9]:
pho4_x = [pho4_seqToLabel[seq] for seq in seqs]
pho4_bins = np.linspace(np.min(pho4_x),np.max(pho4_x), 50)
cbf1_x = [cbf1_seqToLabel[seq] for seq in seqs]
cbf1_bins = np.linspace(np.min(cbf1_x),np.max(cbf1_x), 50)
pho4_rmses = {}
pho4_stddevs = {}
pho4_rmses["observed"] = 0
pho4_stddevs["observed"] = np.std(pho4_x)
cbf1_rmses = {}
cbf1_stddevs = {}
cbf1_rmses["observed"] = 0
cbf1_stddevs["observed"] = np.std(cbf1_x)
for key in pho4_seqToPred:
    print(key)

    #  Pho4
    y = [pho4_seqToPred[key][seq] for seq in seqs]
    metadata = {}
    metadata["Obs. mean"] = np.mean(pho4_x)
    metadata["Obs. stddev"] = np.std(pho4_x)
    metadata["Pred. mean"] = np.mean(y)
    metadata["Pred. stddev"] = np.std(y)
    pho4_stddevs[key] = np.std(y)
    pho4_rmses[key] = math.sqrt(mean_squared_error(pho4_x, y))
    plt.figure()
    matplotlib.rc('font', **font)
    plt.hist(pho4_x, pho4_bins, alpha=0.5, density=True, label='Observed')
    plt.hist(y, pho4_bins, alpha=0.5, density=True, label='Predicted')
    plt.legend()
    plt.title(key+"_pho4")
    plt.savefig('comparison_figs/dynamic_range/'+key+'_pho4.png', dpi=300, format='png')
    with open('comparison_figs/dynamic_range/'+key+'_pho4_metadata.json', 'w') as fp: json.dump(metadata, fp)
    plt.clf()
    
    #  Cbf1
    y = [cbf1_seqToPred[key][seq] for seq in seqs]
    metadata = {}
    metadata["Obs. mean"] = np.mean(cbf1_x)
    metadata["Obs. stddev"] = np.std(cbf1_x)
    metadata["Pred. mean"] = np.mean(y)
    metadata["Pred. stddev"] = np.std(y)
    cbf1_stddevs[key] = np.std(y)
    cbf1_rmses[key] = math.sqrt(mean_squared_error(cbf1_x, y))
    plt.figure()
    matplotlib.rc('font', **font)
    plt.hist(cbf1_x, cbf1_bins, alpha=0.5, density=True, label='Observed')
    plt.hist(y, cbf1_bins, alpha=0.5, density=True, label='Predicted')
    plt.legend()
    plt.title(key+"_cbf1")
    plt.savefig('comparison_figs/dynamic_range/'+key+'_cbf1.png', dpi=300, format='png')
    with open('comparison_figs/dynamic_range/'+key+'_cbf1_metadata.json', 'w') as fp: json.dump(metadata, fp)
    plt.clf()

distill
distill_1seq
distill_2seq
distill_5seq
distill_10seq
distill_20seq
distill_50seq
weeder
streme
modisco


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [10]:
labels = ['distill_1seq', 'distill_2seq', 'distill_5seq', 'distill_10seq',
          'distill_20seq', 'distill_50seq', 'distill']
stddevs = [pho4_stddevs[key] for key in labels]
rmses = [pho4_rmses[key] for key in labels]

x = [1,2,5,10,20,50,100]  # the label locations

fig, ax = plt.subplots()
matplotlib.rc('font', **font)
plt.plot(x, stddevs, label='Std. Dev.')
plt.plot(x, rmses, label='RMSE')

ax.set_ylabel('Log signal')
ax.set_ylim((0.35, 0.7))
ax.set_xscale('log')
ax.legend()

fig.tight_layout()
plt.savefig('comparison_figs/dynamic_range/num_backgrounds_pho4.png', dpi=300, format='png')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [11]:
labels = ['streme', 'modisco', 'weeder', 'distill', 'observed']
stddevs = [pho4_stddevs[key] for key in labels]
rmses = [pho4_rmses[key] for key in labels]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
matplotlib.rc('font', **font)
rects1 = ax.bar(x - width/2, stddevs, width, label='Std. Dev.')
rects2 = ax.bar(x + width/2, rmses, width, label='RMSE')

ax.set_ylabel('Log signal')
ax.set_ylim((0.35, 0.8))
ax.set_xticklabels(['']+labels)
ax.legend()

fig.tight_layout()
plt.savefig('comparison_figs/dynamic_range/bars_pho4.png', dpi=300, format='png')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [12]:
stddevs = [cbf1_stddevs[key] for key in labels]
rmses = [cbf1_rmses[key] for key in labels]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
matplotlib.rc('font', **font)
rects1 = ax.bar(x - width/2, stddevs, width, label='Std. Dev.')
rects2 = ax.bar(x + width/2, rmses, width, label='RMSE')

ax.set_ylabel('Log signal')
ax.set_ylim((0.5, 1.2))
ax.set_xticklabels(['']+labels)
ax.legend()

fig.tight_layout()
plt.savefig('comparison_figs/dynamic_range/bars_cbf1.png', dpi=300, format='png')
plt.clf()

<Figure size 432x288 with 0 Axes>